Summary:

1. prepare datasets:
	- transformers datasets
	- load_datasets
2. tokenize raw_dataset
	- transformers AutoTokenizer.from_pretrained (bert-base-cased)
	- define tokenize map function
	- declair test, eval data
3. model
	- AutoModelForSequenceClassification (bert-base-cased)
	- training arguments using TrainingArguments from trainsformers
	- using numpy define compute_metrics function for evaluation of our model
	- create a "trainer" = Trainer with specific model, argument, tran&eval data


Finally run trainer.train() to train/fine-tune our model


In [ ]:
#preparing datasets
!pip install transformers datasets


from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

raw_datasets = load_dataset("imdb")
#The raw_datasets object is a dictionary with three keys:
# "train", "test" and "unsupervised"


In [ ]:
#for our purpuse we will use BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tk_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation = True)

tokenized_datasets = raw_datasets.map(tk_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]



In [ ]:
#Fine-tuning in PyTorch with the Trainer API

#define our model from pretrained Transformer models
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2)

#instantiate TrainingArguments/hyperparameters
#evaluation_strategy="epoch" enables regular evaluations of 'trainer'
t_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

# since there is not evaluation of how our model preformed by default in our "Trainer",
# we are going to manually compute these matrics

!pip install np
import numpy as np
from datasets import load_metric

#metric = load_metric("accuracy")

#def compute_metrics(eval_pred):
#  logits, labels = eval_pred
#  predictions = np.argmax(logits, axis=-1)
#  return metric.compute(predictions=predictions, references=labels)

#create a trainer
trainer = Trainer(
    model=model,
    args=t_args, 
    train_dataset=small_train_dataset, 
    eval_dataset=small_eval_dataset,
    #compute_metrics=compute_metrics,
)

#to fine-tune our model, we just call the following command using our small_trainig data
trainer.train()
trainer.evaluate()
